TK 07-17-2019
## To Do: 
- Test the means of image generation and validation array
- Increate resolution to 224x224 to fit the model
   - Risk
     - lower if we extrapolate because the neural net should learn and compinsate for it
     - also gives us the option of testing different means of extrapolating

In [2]:
# Data used can be found at:
# https://www.kaggle.com/c/facial-keypoints-detection/data

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from PIL import Image
import seaborn as sns
import keras as K # importing this just in case
from VGG16 import VGG16_Obj # contains the keras models I made
from TrainImageObj import TrainImage # contains some viewing and V&V data 
# object used to debug and evaluate imported images

import tensorflow as tf
import os 
import sys
from datetime import datetime



####################################### definitions for data import#######################################
cwd = os.getcwd()
test = False # just incase there is something we need to test in this
start = datetime.now()
masterStart = datetime.now()

projectDirectory = "C:\\Users\\Ted\\Projects\\Python\\FacialKeypointsDetection"
dataDir = "E:\\TK_PracticeDatabases\\facial-keypoints-detection"
os.chdir(dataDir)
testData = pd.read_csv("test.csv")
trainingData = pd.read_csv("training.csv")
idLookupData = pd.read_csv("IdLookupTable.csv")
sampleSubData = pd.read_csv("SampleSubmission.csv")

# borrowed this ine from Karan Jakhar's post
b4NanFill = trainingData.isnull().any().value_counts() 
trainingData.fillna(0, inplace=True) # replace nans with '0' 
afterNanFill = trainingData.isnull().any().value_counts()
#"Big thank you to Karan Jakhar's post for these lines \n\nB4Nan Handling :  \n"
print(str(b4NanFill) + " \n\nHandling After: \n" + str(afterNanFill) +"\n")
del b4NanFill
del afterNanFill # don't need to delete these to free up memory but doesn't hurt


#########################################################
# Need to test TK 7/17/2019
dataInput = TrainImage(trainingData, 0) # 0 does not matter, the obj gives us all images as well
trainingImageMatrix = dataInput.trainingImageMatrix
validationData = dataInput.validationData
#########################################################

os.chdir(cwd)

if test: # in the begining take a look at the training data
    print(trainingData)

importComplete = datetime.now()
print("Import Finished: " + str(importComplete - start))


########################################## Generate the model ##########################################
start = datetime.now()

os.chdir(projectDirectory)
vgg16 = VGG16_Obj()

model = vgg16.modelReducedFC # this is where I import the Transfer Learning Model
# make sure you take a look at the VGG16.py script, this is where the Keras Model takes place

model.summary() # this will give us the VGG16 without the fully connected layers and label the new FCs as sequentional_1
# we are using the VGG16 as a feature extractor and not changing them!!!

importComplete = datetime.now()
print("\nModified Vgg16 Generated: " + str(importComplete - start))

########################################## fit to model ##########################################
start = datetime.now()
#log all the loss data
csvLogging = K.callbacks.CSVLogger("FacialPointExtractor_Loss.log", append=True)

model.compile(optimizer = 'adam', # adam seems to be the best optimizer I've used (mostly for classification tho) 
              loss = 'mse', # mse because we want a continuous number!!!
              metrics = ['mae', 'accuracy']) # print/log some extra info
#'''
model.fit(trainingImageMatrix, validationData   #need data to insert 
          batch_size=256, 
          epochs=50, # more epochs better the prediction until gain is saturated, start small and go bigger if needed
          shuffle=True, # shuffling is never a bad idea
          callbacks=[csv_logger]
          validation_split=0.25 ) # take a 1/4 of the data for validation purposes '''
doneFitting = datetime.now()
print("\Fitting Complete: " + str(doneFitting-start))


C:\Anaconda3\lib\site-packages\h5py\__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


Big thank you to Karan Jakhar's post for these lines 

B4Nan Handling :  
True     28
False     3
dtype: int64 

Handling After: 
False    31
dtype: int64

Import Finished: 0:00:02.973936
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D

In [33]:
trainingImage[2].shape

AttributeError: 'str' object has no attribute 'shape'

In [30]:
validationData = np.array(trainingData)
trainingImage = validationData[:,-1]
validationData = validationData[:,:-1]
print(str(validationData.shape), str(trainingImage.shape))

(7049, 30) (7049,)


In [34]:
len(trainingData)

7049

In [12]:
trainingData

,left_eye_center_x,left_eye_center_y,right_eye_center_x,right_eye_center_y,left_eye_inner_corner_x,left_eye_inner_corner_y,left_eye_outer_corner_x,left_eye_outer_corner_y,right_eye_inner_corner_x,right_eye_inner_corner_y,...,nose_tip_y,mouth_left_corner_x,mouth_left_corner_y,mouth_right_corner_x,mouth_right_corner_y,mouth_center_top_lip_x,mouth_center_top_lip_y,mouth_center_bottom_lip_x,mouth_center_bottom_lip_y,Image
0,66.033564,39.002274,30.227008,36.421678,59.582075,39.647423,73.130346,39.969997,36.356571,37.389402,...,57.066803,61.195308,79.970165,28.614496,77.388992,43.312602,72.935459,43.130707,84.485774,238 236 237 238 240 240 239 241 241 243 240 23...
1,64.332936,34.970077,29.949277,33.448715,58.856170,35.274349,70.722723,36.187166,36.034723,34.361532,...,55.660936,56.421447,76.352000,35.122383,76.047660,46.684596,70.266553,45.467915,85.480170,219 215 204 196 204 211 212 200 180 168 178 19...
2,65.057053,34.909642,30.903789,34.909642,59.412000,36.320968,70.984421,36.320968,37.678105,36.320968,...,53.538947,60.822947,73.014316,33.726316,72.732000,47.274947,70.191789,47.274947,78.659368,144 142 159 180 188 188 184 180 167 132 84 59 ...
3,65.225739,37.261774,32.023096,37.261774,60.003339,39.127179,72.314713,38.380967,37.618643,38.754115,...,54.166539,65.598887,72.703722,37.245496,74.195478,50.303165,70.091687,51.561183,78.268383,193 192 193 194 194 194 193 192 168 111 50 12 ...
4,66.725301,39.621261,32.244810,38.042032,58.565890,39.621261,72.515926,39.884466,36.982380,39.094852,...,64.889521,60.671411,77.523239,31.191755,76.997301,44.962748,73.707387,44.227141,86.871166,147 148 160 196 215 214 216 217 219 220 206 18...
5,69.680748,39.968748,29.183551,37.563364,62.864299,40.169271,76.898243,41.171888,36.401047,39.367626,...,58.800000,64.869084,82.471178,31.990430,81.669084,49.308112,78.487626,49.432374,93.898766,167 169 170 167 156 145 106 68 52 24 20 15 21 ...
6,64.131866,34.290040,29.578953,33.138040,57.797154,35.154040,69.026577,34.290040,34.761664,33.714040,...,53.006174,55.781799,70.570953,33.898309,70.282953,45.127732,65.099597,45.415732,78.057020,109 109 125 141 145 139 120 81 45 34 31 35 36 ...
7,67.468893,39.413452,29.355961,39.621717,59.554951,40.454773,75.591612,40.038245,37.478214,40.454773,...,56.907961,66.843961,80.025786,30.189204,81.275184,48.100194,71.278602,48.516816,91.063922,178 177 178 179 179 179 181 185 188 184 192 20...
8,65.802880,34.755200,27.475840,36.185600,58.652160,37.329280,72.952960,35.899520,36.342400,37.043200,...,55.348480,65.516800,73.368320,31.194240,74.798080,49.213440,67.648000,49.499520,83.093120,164 158 118 76 66 69 59 53 43 31 35 41 45 55 6...
9,64.121231,36.740308,29.468923,38.390154,58.620923,37.840615,71.272000,37.290338,36.344615,39.490462,...,54.341538,65.771692,69.192615,32.769231,72.217846,49.270154,67.542769,49.545231,76.893538,226 227 225 224 221 220 215 210 210 208 206 18...


In [18]:
def weighted_binary_crossentropy(y_true, y_pred):
    # Oiginal binary crossentropy (see losses.py):
    # K.mean(K.binary_crossentropy(y_true, y_pred), axis=-1)
    # Calculate the binary crossentropy
    
    one_weight = 0.75
    zero_weight = 1.0 - one_weight
    b_ce = K.binary_crossentropy(y_true, y_pred)
    # Apply the weights
    weight_vector = y_true * one_weight + (1. - y_true) * zero_weight
    weighted_b_ce = weight_vector * b_ce
    # Return the mean error
    return K.mean(weighted_b_ce)

# Defining Arcitecture for Neural Net
## OBJ
- __init__
    - defines functions that can be called only 
- First Function
    - VGG16 sequentially network
    - boolean on training CNN layers
    - boolean for using the CNN layers as feature extractor
    - boolean on training Neural Net layers
    - weight to the 1s 
        - if =np.nan then weights are .5 and .5 for weighted X-entropy
    - Output: full architecture
- Second Function
    - Training Layer
    - Output: Fully Trained Model
- Third Function
    - Testing Layer
    - Output: Model to be used on Data
    
# Defining Arcitecture for Neural Net
## OBJ
- __init__
    - defines functions that can be called only 
- First Function
    - K fold cross validation
    - input 2 ints 
        - number of 0s
        - number of 1s
            - if only the first one populated then force weighted k fold to be false
    - number of bins
        - if bins = np.nan, then calculate even number of bins, if %bins  less than bins then add to final  
    - boolean for weighted k fold (evenly distributed) making even 1s in each bin
        - just incase unweighted 0s and 1s
        - separate the input vector into 2
    - Output: bins that are itterable


In [2]:
# save all images that are not labeled and labeled until a nan appears 
# for the first time 

imageDir = "SubplotImages"
os.chdir(imageDir)
for i in range(0,209):#,len(trainingData)): # only need so many (Nans make anything beyond this tricky to say the least)
    patient = TrainImage(trainingData, i) # load the images, labeled and not labeled
    plt.subplot(121)
    plt.imshow(patient.image)
    plt.title(str(i)+" Base")
    plt.subplot(122)
    plt.imshow(patient.labeledImage)
    plt.title(str(i)+" Labeled")
    plt.savefig(str(i)+" Base&Labeled")
    plt.close()
os.chdir(cwd)

C:\Users\Ted\Projects\Python\FacialKeypointsDetection\TrainImageObj.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  trainData["left_eye_center_y"][index] = 0
C:\Users\Ted\Projects\Python\FacialKeypointsDetection\TrainImageObj.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  trainData["left_eye_center_x"][index] = 0
C:\Users\Ted\Projects\Python\FacialKeypointsDetection\TrainImageObj.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  trainData["left_eye_inner_corner_y"][index]

KeyboardInterrupt: 